In [12]:
import os
import numpy as np
from scipy.io import loadmat
import h5py
from tqdm import tqdm
import cv2
from fuel.datasets.hdf5 import H5PYDataset

In [13]:
def preprocess(hwc_bgr_image, size):
    hwc_rgb_image = cv2.cvtColor(hwc_bgr_image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(hwc_rgb_image, (size))
    chw_image = np.transpose(resized, axes=(2, 0, 1))
    return chw_image

In [14]:
def assembleData(imagefolderpath,labelpath):
    
    # extract filenames of images
    filenames = (os.listdir(imagefolderpath))
    filenames.sort()
    
    # extract annotations and sort them
    mat = loadmat(labelpath)
    annotations = mat["annotations"].ravel()
    annotations = sorted(annotations, key=lambda a: str(a[5][0]))
    
    labels = []
    for annotation in annotations:
        label = int(annotation[4])
        labels.append(label)
        
    return filenames,labels

In [15]:
def hdf5Generator(path,filenames,labels,hdf5filepath,split):
    num = len(filenames)
    hdf5 = h5py.File(hdf5filepath, mode="w")
    
    # creating dataset for images
    image_size = (256, 256)
    array_shape = (num, 3) + image_size
    ds_images = hdf5.create_dataset("images", array_shape, dtype=np.uint8)
    ds_images.dims.label = ("batch","channel","height","width")
#     ds_images.dims[0].label = "batch"
#     ds_images.dims[1].label = "channel"
#     ds_images.dims[2].label = "height"
#     ds_images.dims[3].label = "width"
    
    # write images to the hdf5
    for i, filename in tqdm(enumerate(filenames),total=num):
        raw_image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_COLOR)  # BGR image
        image = preprocess(raw_image, image_size)
        ds_images[i] = image
    
    # creating dataset for labels
    targets = np.array(labels, np.int32).reshape(num, 1)
    ds_targets = hdf5.create_dataset("targets", data=targets)
    ds_targets.dims.label = ("batch","class_labels")
#     ds_targets.dims[0].label = "batch"
#     ds_targets.dims[1].label = "class_labels"
    
    # specify the splits (labels 1~split for train, split~total for test)
    test_head = labels.index(split)
    split_train, split_test = (0, test_head), (test_head, num)
    split_dict = dict(train=dict(images=split_train, targets=split_train),test=dict(images=split_test, targets=split_test))
    hdf5.attrs["split"] = H5PYDataset.create_split_array(split_dict)
    hdf5.flush()
    hdf5.close()

In [ ]:
imagefolderpath = "data/cars_train"
labelpath = "data/car_devkit/devkit/cars_train_annos.mat"
hdf5filepath = "data/cars196.hdf5"
split = 99

filenames,labels = assembleData(imagefolderpath,labelpath)
hdf5Generator(imagefolderpath,filenames,labels,hdf5filepath,split)

 35%|███▍      | 2819/8144 [00:22<00:50, 105.21it/s]